### This file has code for Pricing Fixed Income Securities
     1. Vasicek model
     2. CIR model
     3. G2++ model

In [2]:
import numpy as np
from numpy import sqrt, log, sin, cos, exp, mean, repeat, var, mat, floor, flip
from scipy.stats import norm, ncx2 #ncx2 is non central chi-square
from scipy.sparse import diags
import matplotlib.pyplot as pltC
from math import factorial as f
from datetime import date
from pandas import DataFrame
import pandas as pd
import matplotlib.pylab as pylab
import os

In [3]:

params = {'figure.titlesize': 'x-large',
        'legend.fontsize': 'x-large',
          'figure.figsize': (11, 3),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

############# Ques 1.a
"""
This function uses Vasicek model to price 1 dollar zero coupon bond (discount factor)
"""
def priceZero(r0, sigma, k, mean_r, delta_t, T, FV, path):
    
    N =  int(T/delta_t)
    rates = np.zeros((path, N+1))
    rates[:, 0] = r0
    z = np.random.randn(path, N+1)
    for i in range(N):
        rates[:, i+1] = rates[:, i] + k*(mean_r - rates[:, i])*delta_t + sigma*sqrt(delta_t)*z[:, i]
    
    R = delta_t*np.sum(rates[:, 1:N+1], axis=1)
    zeroDiscountFactor = mean(np.exp(-R))
    return zeroDiscountFactor*FV

r0 = 0.05
sigma = 0.10
k = 0.82
mean_r = 0.05
delta_t = 1/360 #time step is one day
T = 0.5 

path = 1000
FV = 1000
price_1a = priceZero(r0, sigma, k, mean_r, delta_t, T, FV, path)





################# Ques 1.b
def priceCoupon(r0, sigma, k, mean_r, delta_t, T, PMT, FV, path):
    vpriceZeroCoupon = np.vectorize(priceZero)
    couponTimes = np.arange(0.5, T+0.1, 0.5)
    priceOneDollar = vpriceZeroCoupon(r0, sigma, k, mean_r, delta_t, couponTimes, 1, path)
    
    payments = np.repeat(PMT, couponTimes.shape[0])
    payments[-1] = PMT+FV
    
    priceofCouponBond = sum(payments*priceOneDollar)
    
    return priceofCouponBond


path = 1000    
r0 = 0.05
sigma = 0.1
k = 0.82
mean_r = 0.05
delta_t = 1/365 #time step is one day
T = 4 
FV = 1000
PMT = 30

price_1b = priceCoupon(r0, sigma, k, mean_r, delta_t, T, PMT, FV, path)





################ Ques 1.c
def callOptionOnZero(r0, sigma, k, r, delta_t, t, T, FV, strike, path):
    n = int(t/delta_t)  # steps till call option expiry
    rates = np.zeros((path, n+1))
    rates[:, 0] = r0
    z = np.random.randn(path, n+1)
    for i in range(n):
        rates[:, i+1] = rates[:, i] + k*(mean_r - rates[:, i])*delta_t + sigma*sqrt(delta_t)*z[:, i]
    
    B=(1-np.exp(-k*(T-t)))/k  #B(t, T)
    A=np.exp( ( mean_r-(sigma**2/(2*k**2)) ) * ( B-(T-t) ) - ((sigma**2)*(B**2)/(4*k)) )
    priceZeroBond_at_t = FV*A*np.exp(-B*rates[:, n])   #P(t, T)
    
    #price of call option
    payOff = priceZeroBond_at_t - strike
    payOff = np.where(payOff>0, payOff, 0)
    
    R = delta_t*np.sum(rates[:, 1:n], axis=1)
    option_price = mean(np.exp(-R)*payOff)
        
    return option_price


path = 1000
r0 = 0.05
sigma = 0.1
k = 0.82
mean_r = 0.05
delta_t = 1/360 #time step is one day
T = 0.5  #Zero bond maturity 
t = 3/12 #Call option expiry
strike = 980
FV = 1000 #bond future value at T

price_1c = callOptionOnZero(r0, sigma, k, mean_r, delta_t, t, T, FV, strike, path)




######################### Ques 1.d
def priceCouponBond_tilOptionExpiry(r0, sigma, k, mean_r, delta_t, t, T, PMT, FV, path):

    couponTimes = np.arange(0.5, T+0.1, 0.5) #semiannual coupon originally
    couponTimes = couponTimes - t            #option is expiring at time t so coupon payment time will closer by t now
    vpriceZeroCoupon = np.vectorize(priceZero)
    priceOneDollar = vpriceZeroCoupon(r0, sigma, k, mean_r, delta_t, couponTimes, 1, path)
    
    payments = np.repeat(PMT, couponTimes.shape[0])
    payments[-1] = PMT+FV
    
    priceofCouponBond = sum(payments*priceOneDollar)
    return priceofCouponBond


def callOptionOnCouponBond(r0, sigma, k, mean_r, delta_t, t, T, FV, PMT, strike, path):

    #simulating interest rate till option expiry at t
    n = int(t/delta_t) #steps till option expiry
    rates = np.zeros((path, n+1))
    rates[:, 0] = r0
    z = np.random.randn(path, n+1)
    for i in range(n):
        rates[:, i+1] = rates[:, i] + k*(mean_r - rates[:, i])*delta_t + sigma*sqrt(delta_t)*z[:, i]
    
    R = delta_t*np.sum(rates[:, 1:n], axis=1) #this will be used to discount option payOff
    
    #Now for each path of interest rate till option expiry 
    #Again simulating multiple paths to calculate bond value P(t,T) 
    #price of coupon bond at t expiring at T
    bondPrice_at_t = np.zeros((path, 1))
    for i in range(path):
        bondPrice_at_t[i, 0] =  priceCouponBond_tilOptionExpiry(rates[i, n], sigma, k, mean_r, delta_t, t, T, PMT, FV, 500)
    
    payOff = bondPrice_at_t - strike
    payOff = np.where(payOff > 0, payOff, 0)
    option_price = mean(exp(-R)*payOff) 
    return option_price


path = 1000    
r0 = 0.05
sigma = 0.1
k = 0.82
mean_r = 0.05
delta_t = 1/360 #time step is one day
T = 4 
FV = 1000
PMT = 30
t = 3/12
strike = 980

price_1d = callOptionOnCouponBond(r0, sigma, k, mean_r, delta_t, t, T, FV, PMT, strike, path)


############ Ques 1.e

    
r0 = 0.05
sigma = 0.1
k = 0.82
mean_r = 0.05
delta_t = 1/360 #time step is one day
T = 4 
FV = 1000
PMT = 30
t = 3/12
K = 980

couponTimes = np.arange(0.5, T+0.1, 0.5)
coupons = np.repeat(PMT, couponTimes.shape[0])
coupons[-1] = FV+PMT

# now calculating price of each coupon(equivalent to zero bond) at time t (option expiry time) 

# calculating T-T_i or T-t 
B=(1-np.exp(-k*(couponTimes-t)))/k  #B(t, T)
A=np.exp( ( mean_r-(sigma**2/(2*k**2)) ) * ( B-(couponTimes-t) ) - ((sigma**2)*(B**2)/(4*k)) )

epsilon = 10**(-10)
rstar = 10**(-10)
pi = A*exp(-B*rstar)
error = sum(coupons*pi)-K

while (error >= epsilon):
    rstar = rstar+0.0001
    pi = A*exp(-B*rstar)
    error = sum(coupons*pi)-K

#new strike price corresponding to each coupon (zero coupon bond for that period)
K_i = A*exp(-B*rstar)

#Now pricing each coupon at time t=0, calculating P(t, Ti)
Bi=(1-np.exp(-k*(couponTimes-0)))/k  #B(t, T)
Ai=np.exp( ( mean_r-(sigma**2/(2*k**2)) ) * ( Bi-(couponTimes) ) - ((sigma**2)*(B**2)/(4*k)) )
P_t_Ti = Ai*exp(-Bi*r0)


#calculating P(t, T) 
B1 = (1/k)*(1-exp((-k)*(t)))
A1 = exp(((mean_r-((sigma**2)/(2*(k**2))))*((B1-(t)))) - ((sigma**2)*(B1**2)/(4*k)))
P_t_T = (A1*exp(-B1*r0))

sigmap = (sigma/k) * (1-exp(-k*(couponTimes-t)))*sqrt( (1-exp(-2*k*t))/(2*k) )

d1 = (1/sigmap)*log(P_t_Ti/(K_i*P_t_T))+.5*sigmap
d2 = d1 - sigmap

c= (P_t_Ti*norm.cdf(d1)) - (K_i * P_t_T * norm.cdf(d2))

price_1e = sum(coupons*c)


In [4]:

def priceZeroCIR(r0, sigma, k, mean_r, delta_t, T, FV, path):
    
    N =  int(T/delta_t)
    rates = np.zeros((path, N+1))
    rates[:, 0] = r0
    z = np.random.randn(path, N+1)
    for i in range(N):
        rates[:, i+1] = rates[:, i] + k*(mean_r - rates[:, i])*delta_t + sigma*sqrt(rates[:, i])*sqrt(delta_t)*z[:, i]
    
    R = delta_t*np.sum(rates[:, 1:N+1], axis=1)
    zeroDiscountFactor = mean(np.exp(-R))
    return zeroDiscountFactor*FV

def callOptionZeroBond_CIR(r0, sigma, k, mean_r, delta_t, t, T, FV, strike, path):

    #simulating interest rate till option expiry at t
    n = int(t/delta_t) #steps till option expiry
    rates = np.zeros((path, n+1))
    rates[:, 0] = r0
    z = np.random.randn(path, n+1)
    for i in range(n):
        rates[:, i+1] = rates[:, i] + k*(mean_r - rates[:, i])*delta_t + sigma*sqrt(rates[:, i])*sqrt(delta_t)*z[:, i]
    
    R = delta_t*np.sum(rates[:, 1:n], axis=1) #this will be used to discount option payOff
    
    #Now for each path of interest rate till option expiry 
    #Again simulating multiple paths to calculate bond value P(t,T) 
    #price of coupon bond at t expiring at T
    bondPrice_at_t = np.zeros((path, 1))
    for i in range(path):
        bondPrice_at_t[i, 0] =  priceZeroCIR(rates[i, n], sigma, k, mean_r, delta_t, T-t, FV, 100)
    
    payOff = bondPrice_at_t - strike
    payOff = np.where(payOff > 0, payOff, 0)
    option_price = mean(exp(-R)*payOff) 
    return option_price

path = 10000    
r0 = 0.05
sigma = 0.12
k = 0.92
mean_r = 0.055
delta_t = 1/360 #time step is one day
T = 1 #bond maturity 
FV = 1000
t = 0.5
strike = 980

price_2a = callOptionZeroBond_CIR(r0, sigma, k, mean_r, delta_t, t, T, FV, strike, path)


######## Ques 2.b ##############
  
r0 = 0.05
sigma = 0.12
k = 0.92
mean_r = 0.055
delta_t = 1/360 #time step is one day
T = 1 #bond maturity 
FV = 1000
t = 0.5 #Option maturity
strike = 980

## interest rate range
rmin = 0
rmax = 0.1
delta_r = 0.001
M = int((rmax-rmin)/delta_r)+1
N = int(t/delta_t)+1 #time steps between 0 and option maturity t
rates = np.linspace(rmax, rmin, M)
#rates = rates.reshape(M, 1)

## price of zero bond CIR Explicit way
h1 = sqrt((k**2)+(2*(sigma**2)))
h2 = (k+h1)/2
h3 = (2*k*mean_r)/sigma**2
A = (  h1* exp(h2*(T-t) )/(h2*(exp(h1*(T-t))-1)+ h1))**h3
B = (exp(h1*(T-t))-1)/(h2*( exp(h1*(T-t)) - 1 ) + h1)
bond_price_at_t = FV*(A*exp(-B*rates))
bond_price_at_t = bond_price_at_t.reshape(M, 1)


#here sigma=sigma*sqrt(rate) and nu=k(mean_r-rate)
i = np.arange(1, M-1, 1) 
Pu = -0.5*delta_t * ( sigma**2*rates[i]/delta_r**2 + k*(mean_r-rates[i])/delta_r )
Pm = 1 + delta_t* ( sigma**2*rates[i]/delta_r**2 + rates[i])
Pd = -0.5*delta_t * ( sigma**2*rates[i]/delta_r**2 - k*(mean_r-rates[i])/delta_r )
Amat = diags([Pu, Pm, Pd], [0, 1, 2], shape=(M-2, M)).todense()
Amat = np.concatenate((Amat[0], Amat, Amat[M-2-1]), axis=0)
Amat[0, 0:3] = [1, -1, 0]
Amat[M-1, M-3:M] = [0, 1, -1]


F = mat(np.zeros((M, N))) #payoff matrix
F[:, :] = bond_price_at_t-strike  # Call option payoff K-S at maturity
F = np.where(F>0, F, 0)

Bmat = np.zeros((M, 1))        

for i in range(N-2, -1, -1):
    Bmat[:, 0] = F[:, i+1]
    Bmat[0, 0] = rates[0] - rates[1]
    Bmat[M-1, 0] = 0
    mat(F)[:, i] = np.linalg.inv(mat(Amat))*Bmat
    
price_2b = np.interp(r0, flip(rates, axis=0), flip(F[:, 0], axis=0))


################## 2.c ################
r0 = 0.05
sigma = 0.12
k = 0.92
mean_r = 0.055
delta_t = 1/360 #time step is one day
FV = 1000
t = 0.5 #Option maturity
strike = 980
S = 1 #bond maturity 
T = 0.5 #Option maturity
t = 0 # find price at time 0

K = 980/1000 #formula is for $1 bond (will multiply by srike price in the last line)


### Price of bond P(t, S); t=0
h1 = sqrt( (k**2)+(2*(sigma**2)) )
h2 = (k+h1)/2
h3 = (2*k*mean_r)/sigma**2

A1 = (h1*exp(h2*(S))/(h2*(exp(h1*(S))-1)+ h1))**h3
B1 = (exp(h1*(S))-1)/(h2*( exp(h1*(S)) - 1 ) + h1)
P_t_S = A1*exp(-B1*r0)

### Price of bond P(t, T); t=0
A2 = (h1*exp(h2*(T))/(h2*(exp(h1*(T))-1)+ h1))**h3
B2 = (exp(h1*(T))-1)/(h2*( exp(h1*(T)) - 1 ) + h1)
P_t_T = A2*exp(-B2*r0)

A_T_S = (h1*exp(h2*(S-T))/(h2*(exp(h1*(S-T))-1)+ h1))**h3
B_T_S = (exp(h1*(S-T))-1)/(h2*( exp(h1*(S-T)) - 1 ) + h1)

### Explicit Valuation of call option
theta = sqrt( k**2 + 2*sigma**2 )
phi=(2*theta)/( sigma**2 * (exp(theta*(T-t))-1))
psi=(k+theta)/(sigma**2)

r_star = log(A_T_S/K)/B_T_S


x1 = 2*r_star*(phi+psi+B_T_S)
p1 = (4*k*mean_r/(sigma**2))
q1 = (2*(phi**2)*r0*exp(theta*T))/(phi+psi+B)
chi_1 = ncx2.cdf(x1,p1,q1)

x2 = 2*r_star*(phi+psi)
p2 = p1
q2 = (2*(phi**2)*r0*exp(theta*T))/(phi+psi)
chi_2 = ncx2.cdf(x2,p2,q2)


price_2c = FV * P_t_S *chi_1 - (strike * chi_2 * P_t_T)


In [5]:

#function to calculate zero coupon bond price using G2++ method
def priceZeroG2(x0, y0, r0, phi, rho, a, b, sigma, eta, delta_t, S, FV, path):
    N =  int(S/delta_t) #till bond expiry
    
    x = np.zeros((path, N+1))
    y = np.zeros((path, N+1))
    rates = np.zeros((path, N+1))
    x[:, 0] = x0
    y[:, 0] = y0
    rates[:, 0] = r0
    
    z = np.random.randn(2*path, N+1)
    z1 = np.split(z, 2)[0]
    z2 = np.split(z, 2)[1]

    
    for i in range(N):
        x[:, i+1] = x[:, i] - a*x[:, i]*delta_t + sigma*sqrt(delta_t)*z1[:, i]
        y[:, i+1] = y[:, i] - b*y[:, i]*delta_t + eta*( rho*sqrt(delta_t)*z1[:, i] + sqrt(1-rho**2)*sqrt(delta_t)*z2[:, i] )
    
    rates = x + y + phi     
    R = delta_t*np.sum(rates[:, 1:N+1], axis=1)
    zeroDiscountFactor = mean(np.exp(-R))
    return FV*zeroDiscountFactor

#function to calculate eur put option
def eurPut(x0, y0, r0, phi0, rho, a, b, sigma, eta, strike, delta_t, T, S, FV, path):
    N =  int(T/delta_t) # till option expiry
    
    x = np.zeros((path, N+1))
    y = np.zeros((path, N+1))
    rates = np.zeros((path, N+1))
    x[:, 0] = x0
    y[:, 0] = y0
    rates[:, 0] = r0
    
    z = np.random.randn(2*path, N+1)
    z1 = np.split(z, 2)[0]
    z2 = np.split(z, 2)[1]

    
    for i in range(N):
        x[:, i+1] = x[:, i] - a * x[:, i]*delta_t + sigma*sqrt(delta_t)*z1[:, i]
        y[:, i+1] = y[:, i] - b * y[:, i]*delta_t + eta * ( rho*sqrt(delta_t)*z1[:, i] + sqrt(1-rho**2 ) * sqrt(delta_t) * z2[:, i] )
  
    rates = x + y + phi
    R = delta_t*np.sum(rates[:, 1:N], axis=1) #this will be used to discount option payOff
    
    #Now for each path of interest rate till option expiry 
    #Again simulating multiple paths to calculate bond value P(T, S) 
    #price of coupon bond at T expiring at S, t is 0
    bondPrice_at_t = np.zeros((path, 1))
    for i in range(path):
        bondPrice_at_t[i, 0] =  priceZeroG2(x[i, N], y[i, N], rates[i, N], phi, rho, a, b, sigma, eta, delta_t, S-T, FV, 100)
    
    payOff = strike- bondPrice_at_t 
    payOff = np.where(payOff > 0, payOff, 0)
    option_price = mean(exp(-R)*payOff) 
    return option_price


x0 = 0
y0 = 0
phi = 0.03
r0 = 0.03
rho = 0.7 #corr
a = 0.1
b = 0.3
sigma = 0.03
eta = 0.08
strike = 950
T = 0.5
S = 1
FV = 1000
delta_t = 1/360
path = 10000

price_3a = eurPut(x0, y0, r0, phi, rho, a, b, sigma, eta, strike, delta_t, T, S, FV, path)


#################Ques 3 (part b) explicit formula################
x0 = 0
y0 = 0
phi = 0.03
r0 = 0.03
rho = 0.7
a = 0.1
b = 0.3
sigma = 0.03
eta = 0.08
strike = 950
S = 1
T = 0.5
t = 0
FV = 1000

#function to generate V
def priceZero_G2PlusPlus_Explicit(a,b,sigma,eta,T,rho, phi):
    V1 = ((sigma**2)/a**2) * (T + (2*exp(-a*T)/a) - (exp(-2*a*T)/(2*a)) - (3/(2*a)))
    V2 = ((eta**2)*(T + (2*exp(-b*T)/b) - (exp(-2*b*T)/(2*b)) - (3/(2*b))))/(b**2)
    V3 = ((2*rho*sigma*eta)*(T + ((exp(-a*T) - 1)/a) + ((exp(-b*T) - 1)/b) - ((exp(-(a+b)*T) - 1)/(a+b))))/(a*b)
    V = V1 + V2 + V3
    PriceZeroCouponOneDollar = exp((-phi*T) - (((1- exp(-a*T))/a)*x0) - (((1- exp(-b*T))/b)*y0) + (V/2))
    return (PriceZeroCouponOneDollar)

P_t_T = priceZero_G2PlusPlus_Explicit(a,b,sigma,eta,T,rho, phi)
P_t_S = priceZero_G2PlusPlus_Explicit(a,b,sigma,eta,S,rho, phi)


sigma1 = ((sigma**2)*((1- exp(-a*(S-T)))**2) * (1 - exp(-2*a*T)))/(2*(a**3))
sigma2 = ((eta**2)*((1- exp(-b*(S-T)))**2) * (1 - exp(-2*b*T)))/(2*(b**3)) 
sigma3 = ((2*rho*sigma*eta)*(1- exp(-a*(S-T)))*(1- exp(-b*(S-T)))*(1- exp(-T*(a+b))))/(a*b*(a+b))
sigma_square = sigma1 + sigma2 + sigma3

sig = sqrt(sigma_square)
K = strike/1000
d1 = ((log((K*P_t_T)/(P_t_S)))/sig) - (sig/2)
d2 = ((log((K*P_t_T)/(P_t_S)))/sig) + (sig/2)
Price_3b = -FV*P_t_S*norm.cdf(d1) + P_t_T*strike*norm.cdf(d2)

